In [11]:
import requests
from bs4 import BeautifulSoup
import re

# API tests

In [83]:
class MAL_API:

    USERS_ANIMELIST_URL = 'https://api.myanimelist.net/v2/users/{}/animelist?fields=list_status&limit=100'
    # Lists with keys from animelist dictionaries of variables to be extracted.
    list_entry_node_keys = ['id', 'title']
    list_entry_list_status_keys = ['status', 'score', 'num_episodes_watched', 'updated_at']

    def __init__(self, client_id: str) -> None:
        self.CLIENT_ID = client_id
        self.api_headers = {
            'X-MAL-CLIENT-ID' : self.CLIENT_ID
        }

    # TODO thrash this - not used
    def get_anime_ranking(self, request_parameters='?ranking_type=all') -> dict:
        response = requests.get('https://api.myanimelist.net/v2/anime/ranking' + request_parameters, headers=self.api_headers)
        return response.json()
    
    def __parse_list_entry(self, entry: dict) -> dict:
        """Parse an entry of animelist. Select important variables and merge dictionaries.

        Args:
            entry (dict): Anime entry in a animelist from the API.

        Returns:
            dict: Parsed entry of animelist as a single dictionary.
        """
        node = entry['node']
        node_parsed = {key: node[key] for key in self.list_entry_node_keys}
        list_status = entry['list_status']
        list_status_parsed = {key: list_status[key] for key in self.list_entry_list_status_keys}
        # Not all entries have start and/or finished dates. Users have not started or finished some of the anime on their lists.
        # If they don't exist add them with `None` values.
        if 'start_date' in list_status.keys():
            list_status_parsed['start_date'] = list_status['start_date']
        else:
            list_status_parsed['start_date'] = None
        if 'finish_date' in list_status.keys():
            list_status_parsed['finish_date'] = list_status['finish_date']
        else:
            list_status_parsed['finish_date'] = None
        parsed_animelist = {**node_parsed, **list_status_parsed}
        return parsed_animelist

    def get_users_animelist(self, username: str) -> dict:
        # End condition response['paging'] - no 'next' key in dictionary.
        animelist_url = self.USERS_ANIMELIST_URL.format(username)
        animelist = []
        end_of_list = False
        while not end_of_list:
            animelist_page = requests.get(animelist_url, headers=self.api_headers).json()
            # Add page to animelist list.
            animelist.extend([self.__parse_list_entry(anime_entry) for anime_entry in animelist_page['data']])
            # Change url to next page. Checking for end condition.
            paging = animelist_page['paging']
            if 'next' in paging.keys():
                animelist_url = paging['next']
            else:
                end_of_list = True
        return animelist
        

CLIENT_ID = '32df078edb7cabcd2eb77f026384e66b'
mal_api = MAL_API(client_id=CLIENT_ID)

In [84]:
tet = mal_api.get_users_animelist('mateuszvaper')

In [89]:
tet[-2]

{'id': 23283,
 'title': 'Zankyou no Terror',
 'status': 'plan_to_watch',
 'score': 0,
 'num_episodes_watched': 0,
 'updated_at': '2021-10-30T15:56:14+00:00',
 'start_date': None,
 'finish_date': None}

In [3]:
api_url = 

response = requests.get(api_url, headers=headers)
resp = response.json()
resp

NameError: name 'headers' is not defined

# Scraping

I was considering supplying usernames with all members from top 100 anime. However a list of members of an anime only contains 7500 users, that have recently updated a this anime on their list. Therefore I will only use the random sample from the `users.php` page of recently active users.

## Scraping usernames by users page with recently active users

In [20]:
users_page_url = 'https://myanimelist.net/users.php'
user_names_filename = 'user_names.txt'

with open(user_names_filename, 'a') as usernames_file:
    # TODO add loop with number of usernames to scrape
    users_soup = BeautifulSoup(requests.get(users_page_url).text, 'html.parser')
    user_names = [user_soup.find('a').get_text() for user_soup in users_soup.find('td').find_all('td')]
    for username in user_names:
        print(username, file=usernames_file)

## Collecting users anime lists